In [2]:
import pandas as pd
import json

In [3]:
bwd_test_log = "logs/parsed_test_bwd.log"
fwd_test_log = "logs/parsed_test_fwd.log"

In [4]:
raw_bwd_logs = [json.loads(line.strip()) for line in open(bwd_test_log).readlines()]

In [5]:
df = pd.DataFrame(raw_bwd_logs).drop("is_interpreter", axis=1)

In [60]:
group_keys = [k for k in df.columns if "DEBUG" not in k]
run_config_keys = [k for k in group_keys if not any(x in k for x in ["passed", "diff"])]
bwd_key_pats = ["dx", "dB", "dC", "dh"]
fwd_key_pat = ["return_", "y2_", "y_", "T_", "T_hat"]
common_keys = [k for k in run_config_keys if not any(x in k for x in bwd_key_pats + fwd_key_pat)]
bwd_group_keys = [k for k in group_keys if not any(x in k for x in fwd_key_pat)]
fwd_group_keys = [k for k in group_keys if not any(x in k for x in bwd_key_pats)] 

In [61]:
#Bwd tests sliced by TMA
bwd_tests = df[bwd_group_keys]
bwd_tests_TMA = bwd_tests[bwd_tests.USE_TMA]
bwd_tests_NO_TMA = bwd_tests[~bwd_tests.USE_TMA]

In [63]:
# These are fwd pass in e2e fwd / bwd tests
fwd_tests = df[df.y_diff.notna()][fwd_group_keys]
fwd_tests_TMA = fwd_tests[fwd_tests.USE_TMA]
fwd_tests_NO_TMA = fwd_tests[~fwd_tests.USE_TMA]

In [70]:
# Forward failures
y_failures = fwd_tests[fwd_tests.y_passed == False].sort_values("y_diff", ascending=False)[common_keys + ["y_diff"]]

# Backward failures
dx_failures = bwd_tests[bwd_tests.dx_passed == False].sort_values("dx_diff", ascending=False)[common_keys + ["dx_diff"]]
dB_failures = bwd_tests[bwd_tests.dB_passed == False].sort_values("dB_diff", ascending=False)[common_keys + ["dB_diff"]]
dC_failures = bwd_tests[bwd_tests.dC_passed == False].sort_values("dC_diff", ascending=False)[common_keys + ["dC_diff"]]
dh_failures = bwd_tests[bwd_tests.dh_passed == False].sort_values("dh_diff", ascending=False)[common_keys + ["dh_diff"]]

In [75]:
assert sum(y_failures.dtype == "float32") == 0
assert sum(dx_failures.dtype == "float32") == 0
assert sum(dB_failures.dtype == "float32") == 0
assert sum(dC_failures.dtype == "float32") == 0
assert sum(dh_failures.dtype == "float32") == 0